In [1]:
%load_ext autoreload
%autoreload 2

from config.max_model import ModelConfig
from functional import seq
from models import MaxModel
import numpy as np
from pathlib import Path
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, MultiProcessDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

vgg_loc = "./data/vgg_16.ckpt"
# log 存放位置
#     log 中包含 Inferencer 给出的 metrics， 可用tensorboard查看
#     命令 tensorboard --logdir XXXX --port XXXX
#     类似 jupyter notebook，tensorboard 需要通过浏览器使用， 请建立 SSH tunnel
#     log_dir 下也会存放 ModelSaver 生成的 checkpoint文件，这些文件占用空间很大，
#     建议将 log_dir 放在 /data 分区下，或建立软链接
log_dir = 'train_log/vgg_max_model/'
logger.set_logger_dir(log_dir)

[1027 14:25:56 @logger.py:94] WRN Log directory train_log/vgg_max_model/ exists! Please either backup/delete it, or use a new directory.
[1027 14:25:56 @logger.py:96] WRN If you're resuming from a previous run you can choose to keep it.
[1027 14:25:56 @logger.py:97] Select Action: k (keep) / b (backup) / d (delete) / n (new) / q (quit):
d
[1027 14:25:58 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-d10c0982-6dae-46d2-bb16-00ea9e1ccd1b.json


In [2]:
# 可以在这里修改配置
config = ModelConfig()
# 新增：（width, height) 图片大小。DataManager需要知道图片的大小。这里也可以进行resize操作
config.image_size = (320, 128)
# 新增：划分比例
config.proportion = {'train': 0.6, 'val': 0.2, 'test': 0.2}
# 新增: 在划分数据集时，允许比例的误差
config.tolerance_margin = 0.02
# 新增：在划分数据集时，是否打乱顺序
#    设置为 True，每次划分的结果会不同
#    不过现在我把随机数种子固定了，无论true 或 false每次划分的结果都一样
#    之后可能会取消固定随机数种子
config.shuffle_separation = True
# 新增：要使用哪些方向
config.directions = ['ventral', 'dorsal', 'lateral']
# 新增：DataManager 需要知道 batch_size
config.batch_size = 20
# 为了兼容 DataManager， 我把 stage_allowed 改为 stages 
config.stages = [6]
# 同样的， top_k_labels 改为 annotation_number
config.annotation_number = 10
# max_img 改为 max_sequence_length
config.max_sequence_length = 10
# 请修改为图片所在位置
config.image_directory = str(Path.home()) + \
    "/Documents/flyexpress/DL_biomedicine_image/data/pic_data/"

# standard_images.csv 与 standard_annotations.csv 附带在 repo 的 data 目录下
config.image_table_location = str(Path.home()) + \
    "/Documents/flyexpress/DL_biomedicine_image/data/standard_images.csv"
config.annotation_table_location = str(Path.home()) + \
    "/Documents/flyexpress/DL_biomedicine_image/data/standard_annotations.csv"

In [3]:
# 从vgg checkpoint 回复权值时，要忽略的tensor名称
ignore_restore = ['learning_rate', 'global_step']
# 在训练中，会保存一个某个metric最大（或最小）的模型，用于之后测试
save_name = "max-micro_auc.ckpt"
# 将 probability 转化为 0， 1 的阈值
threshold = 0.5
# 在验证集与测试集上要计算哪些 metrics
validation_metrics = ['mean_average_precision', 'macro_auc', 'micro_auc',
                      'macro_f1', 'micro_f1', 'ranking_mean_average_precision',
                      'coverage', 'ranking_loss', 'one_error']


In [ ]:
data_manager = DataManager.from_config(config)
print(data_manager.get_imbalance_ratio())
train_data = data_manager.get_validation_stream()
val_data = data_manager.get_test_stream()
test_data = data_manager.get_test_stream()

Group numbers:
train: 1476, validation: 449, test: 527
Image numbers:
train: 8489, validation: 2594, test: 3087
                                         train       val      test
embryonic midgut                      1.044321  0.995556  1.125000
ventral nerve cord                    1.748603  1.823899  1.977401
embryonic brain                       1.860465  1.841772  2.028736
embryonic hindgut                     2.400922  2.277372  2.513333
embryonic dorsal epidermis            2.904762  2.741667  2.875000
embryonic/larval muscle system        3.146067  2.870690  2.932836
embryonic central nervous system      3.193182  3.235849  3.543103
embryonic ventral epidermis           3.405970  3.235849  3.319672
embryonic head epidermis              4.698842  4.907895  4.547368
dorsal prothoracic pharyngeal muscle  5.201681  4.831169  4.988636


In [ ]:
model = MaxModel(config)
tf.reset_default_graph()
train_config = TrainConfig(model=model, dataflow=train_data,
                           callbacks=[
                               # 在这里设置 learning rate
                               ScheduledHyperParamSetter('learning_rate', [(0, 1e-4), (20, 1e-4)]),
                               InfRunner(val_data, [AggregateMetric(validation_metrics, threshold)],
                                         [0, 1]),
                               ModelSaver(max_to_keep=5),
                               # 可以将micro_auc 改为其他的
                               MaxSaver('micro_auc', save_name),
                           ],
                           session_init=SaverRestore(
                               model_path=vgg_loc, ignore=ignore_restore),
                           # 跑多少个 epoch
                           max_epoch=2, nr_tower=2)
Trainer(train_config).train()

[1027 14:26:00 @inference_runner.py:83] InferenceRunner will eval on an InputSource of size 65
[1027 14:26:01 @input_source.py:178] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...
[1027 14:26:01 @input_source.py:459] Setting up StagingArea for GPU prefetching ...
[1027 14:26:01 @training.py:41] Training a model of 2 towers
[1027 14:26:01 @training.py:92] Building graph for training tower 0 on device LeastLoadedDeviceSetter-/gpu:0...
[1027 14:26:01 @regularize.py:108] Add REGULARIZATION_LOSSES of 7 tensors on the total cost.
[1027 14:26:01 @training.py:92] Building graph for training tower 1 on device LeastLoadedDeviceSetter-/gpu:1...
[1027 14:26:01 @regularize.py:108] Add REGULARIZATION_LOSSES of 7 tensors on the total cost.
[1027 14:26:02 @model_utils.py:47] Model Parameters: 
name                               shape                 dim  device
---------------------------------  ----------------  -------  -------------
adaption/conv1/kernel:0            [3, 3, 51

100%|##########|56/56[00:25<00:00, 2.19it/s]

[1027 14:26:32 @base.py:267] Epoch 1 (global_step 56) finished, time:25.58 sec.



  0%|          |0/65[00:00<?,?it/s]/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
 15%|#5        |10/65[00:20<00:07, 7.07it/s]

In [ ]:
#np.set_printoptions(formatter={'float_kind': lambda x: '%.3f' % x})
model = MaxModel(config)
tf.reset_default_graph()
pred_config = PredictConfig(model=model,
                            session_init=SaverRestore(
                                model_path=log_dir + save_name),
                            output_names=['logits_export', 'label'],
                            )
pred = SimpleDatasetPredictor(pred_config, test_data)

accumulator = seq(pred.get_result()) \
    .smap(lambda a, b: (a.shape[0], calcu_metrics(a, b, config.validation_metrics, threshold))) \
    .aggregate(Accumulator(*config.validation_metrics), lambda accu, args: accu.feed(args[0], *args[1]))
metrics = accumulator.retrive()
print(metrics)